In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from colour_demosaicing import demosaicing_CFA_Bayer_bilinear
from os import sep
from skimage import io
from kornia.color import ycbcr_to_rgb

from network import DeepISP
from old.test_net import testnet
from dataset import get_data
from loss import deepISPloss

from utils import *

In [ ]:
# data_path = '/home/jupyter/mnt/datasets/S7Dataset/S7-ISP-Dataset'
data_path = '/home/tima/projects/isp/dataset/S7-ISP-Dataset'
crop_size = 400
epoch = 350
norm_mode = 'simple'
train, test = get_data(data_path, target='m', batch_size=1, crop_size=crop_size, flip=None, norm_mode=norm_mode)

print(f'train batch number {len(train)}')
print(f'test  batch number {len(test)}')

In [ ]:
try:
    cp = get_cp_name(epoch)
    params = cp.split(sep)[-1].split('_e')[0][5:].split('-')
    # print(cp)
    model = DeepISP(*[int(i) for i in params]) # type(torch.float)
    cp = torch.load(cp)
    model.load_state_dict(cp['model_state_dict'])
except IndexError:
    model = DeepISP(1, 3)

torch.autograd.set_detect_anomaly(True)
criterion = deepISPloss(norm_mode=norm_mode)

In [ ]:
# there are 3 bad shapes in dataset
# 54 69 in train
# 8 in test

n = 69
x, target = train.dataset[n]

img_shape = (x.shape[1], x.shape[2], x.shape[0])
x = x.reshape((1, *x.shape)).float()
target = target.reshape((1, *target.shape)).float()

y = model(x)

loss = criterion(y, target)
print('loss', loss)

norm = Norm(mode='positive')
y = norm(y, bounds_before=(-1, 1))
x = norm(x, bounds_before=(-1, 1))
target = norm(target, bounds_before=(-1, 1))

# print('x     ', x.shape, x.min(), x.max())
# print('target', target.shape, target.min(), target.max())
# print('y     ', y.shape, y.min(), y.max())

x = x[0].permute(1, 2, 0)
target = target[0].permute(1, 2, 0)
y = y[0].permute(1, 2, 0).detach().numpy()

y = target

In [ ]:
fig, axs = plt.subplots(2, 4)
axs[0, 0].imshow(x[:,:,0])
axs[0, 0].set_title('target R')
axs[0, 1].imshow(x[:,:,1])
axs[0, 1].set_title('target G')
axs[0, 2].imshow(x[:,:,2])
axs[0, 2].set_title('target B')
axs[0, 3].imshow(x)
axs[0, 3].set_title('target')
axs[1, 0].imshow(y[:,:,0])
axs[1, 0].set_title('output R')
axs[1, 1].imshow(y[:,:,1])
axs[1, 1].set_title('output G')
axs[1, 2].imshow(y[:,:,2])
axs[1, 2].set_title('output B')
axs[1, 3].imshow(y)
axs[1, 3].set_title('output')
plt.show()